# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [17]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Output File (CSV)
output_data_file = "../output_data/cities.csv"

# Import API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [18]:
cities_df = pd.read_csv(output_data_file)

cities_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Departamento de Maldonado,90,UY,18-06-2020,93,-34.67,-54.92,57.99,14.99
1,Hushitai,0,CN,18-06-2020,72,41.93,123.51,64.40,4.47
2,Saldanha,0,ZA,18-06-2020,54,-33.01,17.94,57.20,4.70
3,Tanabe,100,JP,18-06-2020,97,33.73,135.37,66.99,11.88
4,Asha,0,RU,18-06-2020,76,55.00,57.27,44.02,4.41


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [19]:
#Data for gmaps. 
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"]
#Value for max intensity.
maxHumidity = cities_df["Humidity"].max()

In [20]:
# Plot Heatmap
fig = gmaps.figure(map_type = 'HYBRID')

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights = humidity, 
                                 dissipating = False, 
                                 max_intensity = maxHumidity, #Scales based on the max humidity of the dataset..
                                 point_radius = 1) 

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
#Narrow down cities to fir our ideal conditions.
hotel_df = cities_df[(cities_df['Max Temp'] >= 80) & (cities_df['Max Temp'] <= 90) & (cities_df['Cloudiness'] <= 10) & (cities_df['Wind Speed'] <= 10)]

hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
28,Rome,1,US,18-06-2020,35,43.21,-75.46,87.80,3.56
56,Buala,2,SB,18-06-2020,76,-8.14,159.59,81.84,3.65
101,Baghdad,0,IQ,18-06-2020,18,33.34,44.40,89.60,9.17
130,Saint George,1,US,18-06-2020,11,37.10,-113.58,89.01,5.82
200,Catuday,10,PH,18-06-2020,80,16.29,119.81,81.32,4.16
232,Charleston,1,US,18-06-2020,74,32.78,-79.93,82.00,9.17
238,Kieta,0,PG,18-06-2020,78,-6.22,155.63,80.47,2.48
272,Presidente Venceslau,7,BR,18-06-2020,39,-21.88,-51.84,80.60,6.93
282,Ganj Dundwāra,2,IN,18-06-2020,50,27.73,78.95,89.73,9.37
443,Ubari,0,LY,18-06-2020,16,26.59,12.78,80.74,6.98


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
#Create empty column to store hotel names
hotel_df['Hotel Name'] = ""

#Filter the columns show only the info we want.
hotel_df = hotel_df[['City','Country','Lat','Lng','Hotel Name']]

hotel_df

/Users/darrenlucca/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Country,Lat,Lng,Hotel Name
28,Rome,US,43.21,-75.46,
56,Buala,SB,-8.14,159.59,
101,Baghdad,IQ,33.34,44.40,
130,Saint George,US,37.10,-113.58,
200,Catuday,PH,16.29,119.81,
232,Charleston,US,32.78,-79.93,
238,Kieta,PG,-6.22,155.63,
272,Presidente Venceslau,BR,-21.88,-51.84,
282,Ganj Dundwāra,IN,27.73,78.95,
443,Ubari,LY,26.59,12.78,


In [23]:
# Set up the base url for the google nearby search.

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterate through pandas dataframe 
for index, row in hotel_df.iterrows(): 
    
    #Set up search parameters for 5km.
    params = {
        "location": f"{row['Lat']},{row['Lng']}",
        "radius" : 50000, #We are not using rank ny distance because we want to rank by importance thus we leave default and use radius only
        "type": "lodging",
        "key": g_key
    }
    
    #Response
    response = requests.get(base_url, params = params).json()
    
    # extract results
    results = response['results']
    
    try: 
        
        #Stores the hotel name under the hotel column.
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

    #If the search cannot find a hotel in 5km radius, try another search for a 10km radius.
    except (KeyError, IndexError):
        
        try:
            
            params = {
                "location": f"{row['Lat']},{row['Lng']}",
                "radius" : 10000, 
                "type": "lodging",
                "key": g_key
            }
            response = requests.get(base_url, params = params).json()
            
            # extract results
            results = response['results']
            
            hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
            
        #If a hotel is not found in a 10km radius, skip the search.
        except (KeyError, IndexError):
            
            print(f"Missing field/result... skipping. {row['City']}")

/Users/darrenlucca/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [24]:
#Removes cities without hotels in the desired radius range.
narrowed_city_df = hotel_df[hotel_df['Hotel Name'] != '']

narrowed_city_df


,City,Country,Lat,Lng,Hotel Name
28,Rome,US,43.21,-75.46,Days Inn by Wyndham Utica
56,Buala,SB,-8.14,159.59,Maringe Lagoon Lodge
101,Baghdad,IQ,33.34,44.40,Baghdad Hotel
130,Saint George,US,37.10,-113.58,Best Western Plus Abbey Inn
200,Catuday,PH,16.29,119.81,Olana Bed & Breakfast
232,Charleston,US,32.78,-79.93,French Quarter Inn
238,Kieta,PG,-6.22,155.63,Arawa Travellers Inn
272,Presidente Venceslau,BR,-21.88,-51.84,K. Hotéis
282,Ganj Dundwāra,IN,27.73,78.95,Taj Hotel
443,Ubari,LY,26.59,12.78,فندق أوباري


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [28]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, 
                             info_box_content = hotel_info)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))